In [1]:
from evaluating_functions import *
from utils import node_complement
from collections import defaultdict

def identify_var_type(G, var_edge):
    assert var_edge in G.variant_edges, f"Input edge {var_edge} is not a variant edge."
    if G.is_inversion(var_edge):
        return 'inversion'
    if G.is_insertion(var_edge):
        return 'insertion'
    if G.is_back_edge(var_edge):
        return 'deletion'
    if G.is_replacement(var_edge):
        return 'replacement'
    if G.is_snp(var_edge):
        return 'snp'
    if G.is_mnp(var_edge):
        return 'mnp'    

In [14]:
def is_back_edge(self, edge: tuple[str, str]) -> bool:
    if self.is_inversion(edge):
        return False
    if self.is_in_tree(edge):
        return False
    branch_point = self.edges[edge]['branch_point']
    return branch_point == edge[1]

def graphs_equal(graph1, graph2):
    """Check if graphs are equal.

    Equality here means equal as Python objects (not isomorphism).
    Node, edge and graph data must match.

    Parameters
    ----------
    graph1, graph2 : graph

    Returns
    -------
    bool
        True if graphs are equal, False otherwise.
    """
    return (
        graph1.adj == graph2.adj
        and graph1.nodes == graph2.nodes
        and graph1.graph == graph2.graph
    )

In [2]:
#AT_csv = pd.read_csv("../Bubble_result_update/bubble_variant_counts_chr1_AT.tsv", sep='\t')
G, walks, walk_sample_names = load_graph_from_pkl("../Graph_obj_chr/chr1.pkl", compressed=False)

In [3]:
#G, walks, walk_sample_names = load_graph_from_pkl("../Graph_obj_chr/chr1.pkl", compressed=False)
G_past, walks, walk_sample_names = load_graph_from_pkl("../Graph_objs/chr1_graph.pkl.gz", compressed=True)

In [4]:
G.reference_path == G_past.reference_path

True

In [15]:
graphs_equal(G.reference_tree, G_past.reference_tree)

False

In [13]:
len(G.variant_edges), len(G_past.variant_edges)

(2418733, 2418734)

In [16]:
len([edge for edge in G.variant_edges if G.is_back_edge(edge)])

0

In [17]:
len([edge for edge in G_past.variant_edges if G_past.is_back_edge(edge)])

9

Example for method comparison

In [11]:
bubble_id = ('4240532', '4240540')
var_list = eval(AT_csv[AT_csv['Bubble_ids'] == "('4240532', '4240540')"]['Within'].iloc[0])
print(bubble_id, var_list)

('4240532', '4240540') {('4240535_-', '4240536_+'), ('4240539_-', '4240540_+'), ('4240537_-', '4240538_+'), ('4240537_+', '4240534_+')}


In [13]:
var_allele_dict = {var:[G.ref_alt_alleles(var)[:2], identify_var_type(G, var)] for var in var_list}
var_allele_dict

{('4240535_-', '4240536_+'): [('T', 'C'), 'snp'],
 ('4240539_-', '4240540_+'): [('T', 'CCTCCCTCCC'), 'replacement'],
 ('4240537_-', '4240538_+'): [('', 'CCTCCCTCC'), 'insertion'],
 ('4240537_+', '4240534_+'): [('C', 'G'), 'snp']}

In [11]:
node_list = [G.positive_node(f"42405{i}_+") for i in range(32, 41)]
node_dict = {node: G.nodes[node]['sequence'] for node in node_list}
node_dict

{'4240532_+': 'C',
 '4240533_+': 'T',
 '4240534_-': 'C',
 '4240535_-': 'C',
 '4240536_+': 'G',
 '4240537_-': 'CCTCCCTCC',
 '4240538_+': 'T',
 '4240539_-': 'C',
 '4240540_+': 'TCCT'}

In [3]:
def write_node_sequence_to_csv(G: PangenomeGraph, filename: str):
    nodes = sorted([node[:-2] for node in list(G.nodes)])
    sequences = [G.nodes[G.positive_node(node+'_+')]['sequence'] for node in nodes]
    df = pd.DataFrame({'NodeID': nodes, 'Sequence': sequences})
    df.to_csv(filename, index=False)

In [4]:
write_node_sequence_to_csv(G, '../Node/chr1.csv')

In [9]:
chr1_allelic_count = pd.read_csv("/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Bubble_result_update/bubble_allele_summary_chr1.tsv", sep='\t')
len({var for var_set in AT_csv['Within'] for var in eval(var_set)}), chr1_allelic_count['allele_count_vcf'].sum()

(2416516, 2196451)